<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/elastic_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elasticsearch で類似検索を実行するサンプル
## 前提条件
このサンプルコードではElasticsearchはv8.13.0を使用します。
またElasticsearchを外部に作成し、そこへアクセスする形となります。

In [ ]:
# Elasticsearch ライブラリのインストール
!pip install elasticsearch
from elasticsearch import Elasticsearch

In [12]:
# Elasticsearchのエンドポイントとポート番号を指定
es_endpoint = "{Elasticsearchのエンドポイント}"
username = "{username} ex. elasticなど"
password = "{password}"

In [ ]:
# Elasticsearchクライアントの作成
try:

    # Elasticsearchクライアントの作成
    es = Elasticsearch(
        hosts=[es_endpoint],
        http_auth=(username,password)
    )

    # Elasticsearchへの接続を確認
    if es.ping():
        print("Elastic Searchへの接続が確認されました。")
    else:
        print("Elastic Searchへの接続に失敗しました。")

except Exception as e:
    print(f"Elastic Searchへの接続中にエラーが発生しました: {e}")

In [ ]:
# Elasticsearchの情報表示
print(es.info())

In [ ]:
# Numpyのインストール
!pip install numpy

In [21]:
# 大量のドキュメントをバイナリ化したNumPyバイナリファイルからデータを読み込む
import numpy as np
vectors = np.load("./embeddings_all.npy")

# インデックス登録

In [26]:
index_name = "my_vector_index"
mapping = {
    "mappings": {
        "properties": {
            "my_vector": {
                "type": "dense_vector",
                "dims": vectors.shape[1]
            }
        }
    }
}

In [ ]:
# インデックス作成
es.indices.create(index=index_name, body=mapping)

In [28]:
# ベクトルデータをElasticsearchにインデックス化
for i, vector in enumerate(vectors):
    doc = {
        "my_vector": vector.tolist()
    }
    es.index(index=index_name, body=doc)

In [24]:
# インデックスの削除関数
def delete_index(es, index_name):
    """
    Elasticsearchのインデックスを削除する関数

    :param es: Elasticsearchクライアントオブジェクト
    :param index_name: 削除するインデックスの名前
    """
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    else:
        print(f"Index '{index_name}' does not exist.")

In [ ]:
# インデックスの削除
# delete_index(es, index_name)

In [ ]:
# 検索対象のテキストのベクトル化(OpenAI の Embeddingを利用)
!pip install openai

In [34]:
from openai import OpenAI
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# クライアントの準備
client = OpenAI()

In [39]:
# テキストを読み込む
txt_file_path = "./法律AIの活用に関する研究報告書.txt"

with open(txt_file_path, "r", encoding="utf-8") as file:
    in_text = file.read()

In [40]:
# 作業対象のファイルをベクトル化する
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-ada-002"
)

# ベクトル化したデータをnumpy配列に変換
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

In [81]:
# ドット積
query = {
  "query": {
    "script_score": {
      "query": {"match_all": {}},
      "script": {
        "source": "dotProduct(params.query_vector, 'my_vector')",
        "params": {"query_vector": in_embeds.flatten().tolist()}
      }
    }
  }
}

In [85]:
# マンハッタン距離(L1ノルム)
query = {
  "query": {
    "script_score": {
      "query": {"match_all": {}},
      "script": {
        "source": "1 / (1 + l1norm(params.query_vector, 'my_vector'))",
        "params": {"query_vector": in_embeds.flatten().tolist()}
      }
    }
  }
}

In [87]:
# ユークリッド距離(L2ノルム)
query = {
  "query": {
    "script_score": {
      "query": {"match_all": {}},
      "script": {
        "source": "1 / (1 + l2norm(params.query_vector, 'my_vector'))",
        "params": {"query_vector": in_embeds.flatten().tolist()}
      }
    }
  }
}

In [89]:
# コサイン類似度のクエリ
query = {
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'my_vector') + 1.0",
                "params": {"query_vector": in_embeds.flatten().tolist()}
            }
        }
    }
}

In [91]:
# knn(k近傍)アルゴリズムでクエリの構築
query = {
    "query": {
        "knn": {
            "field": "my_vector",
            "query_vector": in_embeds.flatten().tolist(),
            "num_candidates": 100
        }
    },
      "size": 5
}

In [83]:
# 実行結果をCSVファイルに出力
import csv
from datetime import datetime

def export_search_results_to_csv(search_result):
    # 現在の日時を取得し、ファイル名に使用
    current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = f"search_results_{current_datetime}.csv"

    # CSVファイルに書き込むためのフィールド名
    field_names = ["doc_id", "score", "vector"]

    # 検索結果をCSVファイルに書き込む
    with open(file_name, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)

        # ヘッダーを書き込む
        writer.writeheader()

        # 検索結果を1行ずつ処理し、CSVファイルに書き込む
        for hit in search_result["hits"]["hits"]:
            doc_id = hit["_id"]
            score = hit["_score"]
            vector = hit["_source"]["my_vector"]

            # CSVファイルに行を書き込む
            writer.writerow({"doc_id": doc_id, "score": score, "vector": str(vector)})

    print(f"Search results exported to {file_name}")

In [92]:
!# 検索結果を取得
search_result = es.search(index=index_name, body=query)
# csvファイルに出力
export_search_results_to_csv(search_result)

# 検索結果を表示
print("Search results:")
for hit in search_result["hits"]["hits"]:
    print("doc Id",hit["_id"])
    print("Score:", hit["_score"])
    print("Vector:", hit["_source"]["my_vector"])
    print("---")

Search results exported to search_results_20240403_083049.csv
Search results:
doc Id -Kauoo4BDbi8FAI_K4ja
Score: 0.98850983
Vector: [0.004213161766529083, -0.017553187906742096, 0.013574641197919846, -0.03077095001935959, -0.00947052612900734, 0.011770416982471943, -3.4980599593836814e-05, 0.015028594993054867, -0.004315599333494902, -0.029925012961030006, -0.007851350121200085, 0.016707250848412514, 0.008413105271756649, -0.008803029544651508, -0.007686128374189138, 0.00528710475191474, 0.03819933161139488, -0.02071223221719265, 0.004622912034392357, -0.01973411813378334, -0.017262397333979607, 0.02013065107166767, -0.022906381636857986, 0.006582445465028286, -0.011235097423195839, 0.007025240454822779, 0.021967919543385506, -0.039388932287693024, 0.006549401208758354, -0.014698151499032974, 0.03460410237312317, -0.009582877159118652, -0.016376806423068047, 0.006453572306782007, -0.0004002503410447389, -0.011750590056180954, 0.009166518226265907, -2.4809120077406988e-05, 0.01334333047